In [1]:
# !/usr/bin/env python
# coding: utf-8

import warnings
warnings.simplefilter("ignore")
from IPython.display import display
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss
from sklearn.calibration import CalibratedClassifierCV
from training_functions_1 import *

In [2]:
path='../outputs/'
optim_results = get_data(name_sav='optimization_results.sav', path=path)

In [3]:
optim_results

hyperparameters       {'max_depth': None, 'max_leaf_nodes': 150, 'mi...
cv_f1                                                          0.273298
cv_precision                                                   0.176069
cv_recall                                                      0.617898
cv_m_c                                                          0.15633
cv_f1_micro                                                    0.620785
cv_precision_micro                                             0.620785
cv_recall_micro                                                0.620785
optimization_date                            2023-08-08 21:35:38.102857
ratio_balance                                                         1
model_name                                       RandomForestClassifier
k_folds                                                               4
test_size                                                          0.25
Name: 0, dtype: object

# 1. AJUSTE FINAL.

El ajuste final de modelo optimizado es realizado por la función __training_model__ contenida en el script __training_functions.py__. Esta función tiene como una de sus entradas la salida de la función __optimization_model__. También, se encarga de:
1. guardar el modelo final en un archivo binario para su consumo,
2. generar las métricas de __testeo__, las cuales se obtienen al aplicar el modelo al conjunto de __testeo__,
3. generar gráficos de interés como el histograma de probabilidades de testeo,
4. calcular las features importances.

Los parámetros que esta función necesita son ya todos conocidos, a excepción de dos:

In [4]:
# Binario para el cálculo de las feature importances.
with_feature_importances=True
# Binario para la construcción del histograma de probabilidades de testeo.
with_probability_density=False

__¡¡¡ADVERTENCIA !!!:__ El cálculo de las feature importances para `HistGradientBoostingClassifier`
tarda horas. Esto debido a que se calculan vía shap-values, puesto que `scikit-learn`no las ofrece nativamente para este estimador.

In [5]:
df_name = 'df_HOMO_CLASS.sav'
features_names =  ['FLAG_OWN_CAR',
                   'FLAG_OWN_REALTY',
                   'NAME_INCOME_TYPE',
                   'NAME_EDUCATION_TYPE',
                   'NAME_FAMILY_STATUS',
                   'NAME_HOUSING_TYPE',
                   'OCCUPATION_TYPE',
                   'FLAG_WORK_PHONE',
                   'FLAG_PHONE',
                   'FLAG_EMAIL',
                   'AGE',
                   'CNT_CHILDREN',
                   'AMT_INCOME_TOTAL',
                   'WORK_YEARS']
objective_name = 'label'
path='../outputs/'

Se procede a ajustar el modelo final.

In [6]:
train_results = training_model(best_hyper_info=optim_results,
                               df_name_sav=df_name,
                               features_names=features_names,
                               objective_name=objective_name,
                               with_feature_importances=with_feature_importances,
                               with_probability_density=with_probability_density,
                               path=path)

2023-08-08 21:51:27 INFO: PROCESO DE ENTRENAMIENTO Y GUARDADO DE MODELO.
2023-08-08 21:51:27 INFO: CONFIGURACIÓN DE DATA.
2023-08-08 21:51:27 INFO: CONFIGURACIÓN GENERALES.
2023-08-08 21:51:27 INFO: TUBERÍAS PARA PROCESAMIENTO Y MUESTREO.
2023-08-08 21:51:27 INFO: AJUSTE FINAL.
2023-08-08 21:51:28 INFO: OBTENCIÓN DE FEATURE IMPORTANCES.
Permutation explainer: 377it [17:08,  2.75s/it]                        


2023-08-08 22:08:38 INFO: FIN DE AJUSTE FINAL.


In [7]:
train_results

hyperparameters            {'max_depth': None, 'max_leaf_nodes': 150, 'mi...
cv_f1                                                               0.273298
cv_precision                                                        0.176069
cv_recall                                                           0.617898
cv_m_c                                                               0.15633
cv_f1_micro                                                         0.620785
cv_precision_micro                                                  0.620785
cv_recall_micro                                                     0.620785
optimization_date                                 2023-08-08 21:35:38.102857
ratio_balance                                                              1
model_name                                            RandomForestClassifier
k_folds                                                                    4
test_size                                                               0.25

# 2. SOBRE LA CALIBRACIÓN DEL MODELO

In [8]:
X_test = pickle.load(open('../outputs/X_test.sav', 'rb'))
y_test = pickle.load(open('../outputs/y_test.sav', 'rb'))
clf = pickle.load(open('../outputs/risk_model.sav', 'rb'))
probs_test = clf.predict_proba(X_test)[:, 1]

In [ ]:
# Calcular el Brier Score
brier_score = brier_score_loss(y_test, probs_test)
print("Brier Score:", brier_score)
logloss = log_loss(y_test, probs_test)
print("Brier Score:", logloss)

In [ ]:
get_calibration_curve(y_test=y_test,
                      probs_test=probs_test)

En efecto, se nota que el modelo está no equilibrado. Se aplicará un ajuste de las probabilidades.

In [ ]:
X_train = pickle.load(open('../outputs/X_train.sav', 'rb'))
y_train = pickle.load(open('../outputs/y_train.sav', 'rb'))

In [ ]:
clf_isotonic = CalibratedClassifierCV(base_estimator=clf, cv='prefit', method='isotonic') #, method='isotonic'
clf_isotonic.fit(X_train, y_train)

In [ ]:
probs_test_cal = clf_isotonic.predict_proba(X_test)[:, 1]

In [ ]:
get_calibration_curve(y_test=y_test,
                      probs_test=probs_test_cal)

In [ ]:
# Calcular el Brier Score
brier_score = brier_score_loss(y_test, probs_test_cal)
print("Brier Score:", brier_score)
logloss = log_loss(y_test, probs_test_cal)
print("Brier Score:", logloss)

Hemos conseguido calibrar el modelo. Veamos cómo efecta esto a las métricas f1, recall y precision de testeo.

In [ ]:
y_pred_uncalibrated = clf.predict(X_test)
y_pred_calibrated = clf_isotonic.predict(X_test)

In [ ]:
f_1 = f1_score(y_test, y_pred_uncalibrated)
recall = recall_score(y_test, y_pred_uncalibrated)
precision = precision_score(y_test, y_pred_uncalibrated)
f_1_cal = f1_score(y_test, y_pred_calibrated)
recall_cal = recall_score(y_test, y_pred_calibrated)
precision_cal = precision_score(y_test, y_pred_calibrated)

In [ ]:
print('f1 modelo Descalibrado: ', f_1)
print('f1 modelo Calibrado: ', f_1_cal)
print('recall modelo Descalibrado: ', recall)
print('recall modelo Calibrado: ', recall_cal)
print('precision modelo Descalibrado: ', precision)
print('precision modelo Calibrado: ', precision_cal)

Vemos que las métrica de recall y precision cambian de manera significativa en el recall y precisión. Habrá que pensar si la calibración del modelo es necesaria para los fines del problema.

# 2. COMPARACIÓN ENTRE TESTEO Y CV

In [ ]:
path='../outputs/'
train_results = get_data(name_sav='final_model_info_RainTomorrow_1.sav', path=path)

Veamos la estabilidad del modelo al comparar las métricas de __cross-validation__ y de __testeo__.

In [ ]:
d_f1 = train_results[['cv_f1', 'final_f1_score']]
d_rec = train_results[['cv_recall', 'final_recall']]
d_prec =  train_results[['cv_precision', 'final_precision']]
d_m_c = train_results[['cv_m_c', 'final_matthews_corrcoef']]
display(d_f1, d_rec, d_prec, d_m_c)

Se puede observar que las métricas son similares entre ellas, lo cual se puede considerar como "una buena señal" y sugiere que el modelo generaliza de manera razonable para datos no vistos.

# 3. SOBRE LAS MÉTRICAS DE TESTEO

Estas son las métricas asociadas al conjunto de __testeo__.

In [ ]:
train_results[['final_recall', 'final_precision',
               'final_accuracy', 'final_f1_score', 'final_roc_auc',
               'final_matthews_corrcoef', 'final_recall_maj', 'final_precision_maj']] 

De las métricas de __testeo__ se dan algunas observaciones:
1. __`final_recall`__ `= 0.776993`, quiere decir que la gran mayoría de las muestras pertenecientes a la clase minoritaria, es decir, "SI" va a llover mañana, son clasificadas correctamente.
2. __`final_precision`__ `= 0.531523`, quiere decir que si bien se localiza correctamente a la gran mayoría de la clase minoritaria, "el precio que hay que pagar" para reconocerlos es el de tener a muchos falsos positivos. De hecho, se puede decir que del total de muestras que el modelo dice que son "SI", en realidad `(1 - 0.531523)% = 46.8447%` NO LO SON.
3. __`final_f1_score`__ `= 0.631234`, quiere decir que hay un equilibrio pobre entre __`recall`__ y __`precision`__. Algo similar puede concluirse observando el __`final_matthews_corrcoef`__.
4.   __`final_recall_maj`__ `= 0.796674` y __`final_precision_maj`__ `= 0.923268`, son las contrapartes de __`recall`__ y __`precision`__ pero para la clase mayoritaria. Naturalmente, a la clase mayoritaria le va bien.
4. __`final_roc_auc`__ `= 0.786834`, es el área bajo la curva ROC. Informa que el modelo clasifica razonablemente bien a la mayoría tanto de la clase minoritaria como de la mayoritaria (observar los respectivos __`recalls`__), pero no toma en cuenta el escenario de desequilibro extremo cuya influencia se ve reflejada en el __`final_precision`__.

# 4. COMENTARIOS FINALES

1. Los scores de decisión, interpretados como "probabliddades", que brindan algunos algoritmos de clasificación como el `HistGradientBoostingClassifier`, muchas veces aún no pueden interpretarse como verdaderas probabilidades. Necesitan pasar por un proceso de ajuste. Siendo así, queda pendiente revisar si el modelo final está __calibrado__. 
2. El "modelo final", más bien el objeto conformado por una __Pipiline__ de procedimientos cuyo paso final es el estimador ajustado, ya tiene integrado preprocesamientos como la codificación de variables categóricas.
3. Pueden consultarse los gráficos obtenidos en la carpeta __outputs/__. Sus nombres son:
    * __final_feature_importances_RainTomorrow_1.html__: exhibe el histograma en orden descendente de las feature importances. Al igual que con la __información mutua__, la variable `Humidity3pm` ocupa el primer lugar de importancia.
    * __final_density_RainTomorrow_1.html__: exhibe el histograma de las probabilidades de testeo.
4. Se decide, por el momento, llevar a producción al modelo final actual a reserva de mejorar su rendimiento, sobretodo enfocando los esfuerzos para mejorar el __f1__ y, más precisamente, el __precision__.